# fpgaConvNet: CNN Model to Vivado IP Example

This tutorial demonstrates the end-to-end flow from a ONNX model to generated hardware. The MPCNN network is used as an example. Let's start by looking at the base-line architecture through the lens of the [fpgaconvnet-model](github.com/AlexMontgomerie/fpgaconvnet-model) modelling part of the project.

In [1]:
from IPython.display import Image 
from fpgaconvnet.models.network import Network

# load network
net = Network("mpcnn", "models/mpcnn.onnx")
net.visualise("baseline.png", mode="png")

# display to jupyter notebook
im = Image('baseline.png')
display(im)

/home/alex/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The baseline network is predicted to have poor performance, as there is no parallelism amongst the modules. We can see what the performance is expected to be for a Zedboard.

In [2]:
# load the zedboard platform details
net.update_platform("platforms/zedboard.json")

# show latency, throughput and resource predictions
print(f"predicted latency (us): {net.get_latency()*1000000}")
print(f"predicted throughput (img/s): {net.get_throughput()} (batch size={net.batch_size})")

predicted latency (us): 7063.0199999999995
predicted throughput (img/s): 141.58249587287025 (batch size=1)


To increase performance, the [samo](github.com/AlexMontgomerie/samo) design-space exploration tool can be used. This tool generalises the CNN to FPGA mapping problem for Streaming Architectures. A command-line interface can be used to optimise a design. We will use the rule-based optimiser, and use a latency performance objective.

In [3]:
!mkdir -p outputs
import samo.cli

# invoking the CLI from python
samo.cli.main([
    "--model", "models/mpcnn.onnx",
    "--platform", "platforms/zedboard.json",
    "--output-path", "outputs",
    "--backend", "fpgaconvnet",
    "--optimiser", "rule",
    "--objective", "latency"
])

#### ( run settings ) ####
 * model    : models/mpcnn.onnx
 * backend  : fpgaconvnet
 * platform : platforms/zedboard.json
 * optimiser : rule
 * objective : latency
 * batch size : 256
 * enable_reconf : true
 * seed : 3917603228
 * output_path : outputs
##########################
Partition 0:
------------

  in (Mbps)    out (Mbps)  total (Mbps)
-----------  ------------  -------------------------
    522.449          8000  8522.448979591834/34112.0


Layer                      Latency (cycles)    DSP    BRAM    LUT     FF
-----------------------  ------------------  -----  ------  -----  -----
sequential/conv1/Conv2D                3136    130     154  14039  17255


  Latency (cycles)  DSP        BRAM       LUT            FF
------------------  ---------  ---------  -------------  --------------
              3136  130 / 220  154 / 280  14039 / 53200  17255 / 106400



Partition 1:
------------

  in (Mbps)    out (Mbps)  total (Mbps)
-----------  ------------  ---------------
    

  in (Mbps)    out (Mbps)  total (Mbps)
-----------  ------------  --------------------------
     104.49      0.612245  105.10204081632654/34112.0


Layer                       Latency (cycles)    DSP    BRAM    LUT     FF
------------------------  ------------------  -----  ------  -----  -----
sequential/conv1/Conv2D                15680     26      50   5391   6556
sequential/pool1/MaxPool                7840      0       6   1806   2614
sequential/conv2/Conv2D                10000    104     132  11026  13714
sequential/pool2/MaxPool                2000      0       3    903   1307
sequential/conv3/Conv2D                10800      4      16   3498   4538
sequential/ip1/MatMul                  13500      4      64  12098  17086
ip2                                     1800      1       4   1347   1888


  Latency (cycles)  DSP        BRAM       LUT            FF
------------------  ---------  ---------  -------------  --------------
             15680  139 / 220  275 / 280  36069 / 

We can look at the optimised network below, as well as the predicted performance. You can see that the optimised network utilises a lot more parallelism, and evidently achieves much better performance.

In [7]:
# load the optimised network
net.load_network("outputs/same.json") # TODO: change name
net.update_partitions()
net.visualise("optimised.png", mode="png")

# print the performance and resource predictions
print(f"predicted latency (us): {net.get_latency()*1000000}")
print(f"predicted throughput (img/s): {net.get_throughput()} (batch size={net.batch_size})")
print(f"predicted resource usage: {net.partitions[0].get_resource_usage()}")

# display to jupyter notebook
im = Image('optimised.png')
display(im)

predicted latency (us): 227.22
predicted throughput (img/s): 4401.0210368805565 (batch size=1)
predicted resource usage: {'FF': 48493, 'LUT': 36738, 'DSP': 144, 'BRAM': 275}


Once we are happy with the design, we can now generate the hardware using the [fpgaconvnet-hls](github.com/AlexMontgomerie/fpgaconvnet-hls) project. This part of the tool acts as a backend for modelling, implementing the hardware building blocks. To start with, we can instantiate the HLS code generator for this hardware configuration.

In [9]:
from fpgaconvnet.hls.generate.network import GenerateNetwork

# create instance of the network
net = GenerateNetwork("mpcnn", "outputs/same.json", "models/mpcnn.onnx")

In [ ]:
import onnx
import numpy as np
from PIL import Image

from fpgaconvnet.hls.generate.network import GenerateNetwork

import fpgaconvnet.tools.onnx_helper as onnx_helper

# create instance of the network
net = GenerateNetwork("single_layer", "single_layer.json", "single_layer.onnx")

# load test data
input_image = Image.open("../mnist_example.png")

# transform image
# input_image = input_image.resize((224, 224), Image.ANTIALIAS)

# to a numpy array
input_image = np.array(input_image, dtype=np.float32)

# add channel dimension
input_image = np.expand_dims(input_image, axis=0)

# normalise
input_image = input_image/np.linalg.norm(input_image)

# duplicate across batch size
input_image = np.stack([input_image for _ in range(1)], axis=0 )

# create project for first partition
net.create_partition_project(0)

# # run the partition's testbench
# net.run_testbench(0, input_image)

# generate hardware
net.generate_partition_hardware(0)

# run co-simulation
net.run_cosimulation(0, input_image)
